## Artificial Neural Network for Customer Churn Prediction
[Notebook from: http://youtube.com/c/ByteSizeDataScience]

How to deal with overfitting.

In [ ]:
import sys
import types
import pandas as pd
import io
import requests

url = 'https://github.com/jacquesroy/byte-size-data-science/raw/master/data/customer_churn.csv'
content = requests.get(url).content
dataset = pd.read_csv(io.StringIO(content.decode('utf-8')))
dataset.head()


In [ ]:
import numpy as np

X = dataset.iloc[:,2:17].values # Columns from Gender on
y = dataset.iloc[:,1].values # CHURN column

### Encoding:
- Categorical: Gender, Status, Car Owner, Paymethod, LocalBilltype, LongDistanceBilltype

Other encoding could be used for some attributes. For example `onehotencoder` for Gender and Status

In [ ]:
# Encoding categorical data before split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
# columns: 0 (Gender), 1(Status), 4 (Car owner), 10 (Payment method), 11 (LocalBillType), 12 (LongDistanceBillType)

labelencoder_X_0 = LabelEncoder()
X[:,0] = labelencoder_X_0.fit_transform(X[:,0])
labelencoder_X_1 = LabelEncoder()
X[:,1] = labelencoder_X_1.fit_transform(X[:,1])
labelencoder_X_4 = LabelEncoder()
X[:,4] = labelencoder_X_4.fit_transform(X[:,4])
labelencoder_X_10 = LabelEncoder()
X[:,10] = labelencoder_X_10.fit_transform(X[:,10])
labelencoder_X_11 = LabelEncoder()
X[:,11] = labelencoder_X_11.fit_transform(X[:,11])
labelencoder_X_12 = LabelEncoder()
X[:,12] = labelencoder_X_12.fit_transform(X[:,12])

In [ ]:
# Since the answer column is "T" or "F", we have to encode it
labelencoder_y = LabelEncoder()
y = labelencoder_y.fit_transform(y)

In [ ]:
# Split dataset
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [ ]:
# Feature scaling
# we need all the values in a standardized range 
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [ ]:
# see: https://keras.io
import keras
from keras.models import Sequential # to initialize NN
from keras.layers import Dense 

In [ ]:
# Create a sequential model (most common in Keras)
classifier = Sequential()

# Create the first hidden layer
classifier.add(Dense(8, activation='relu', input_shape=(15,))) # there are 15 attributes

# Create the second hidden layer
classifier.add(Dense(8, activation='relu'))

# Create the output layer
classifier.add(Dense(1, activation='sigmoid'))

## Model content
Show what the model is using the summary method

In [ ]:
classifier.summary()

## Model visualization
Another way to display the model.

If you get an error, restart the kernel and re-execute all the cells.

In [ ]:
!pip install --user --upgrade pydot
!pip install --user --upgrade graphviz

In [ ]:
from keras.utils.vis_utils import plot_model
from IPython.display import Image

plot_model(classifier, to_file='model_plot.png', show_shapes=True, show_layer_names=True)
Image('./model_plot.png')

In [ ]:
# Compile the neural network
classifier.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
# Train the model (fit)
# batch size: number of records used in each epoch
# history = classifier.fit(X_train, y_train, batch_size=50, epochs=100, validation_split=0.2)
history = classifier.fit(X_train, y_train, batch_size=50, epochs=100, validation_data=(X_test, y_test))

## Training progress
In the previous cell, we passed the validation data so we can look at how the model fares at
each step of the training.

Instead of passing the test data, we could have split the training data into training and validation.
We can plot the accuracy and loss results on a graph that would give us some indications on when 
the model becomes less efficient on the validation data.

We use matplotlib to show these graphics. Note the line:

`%matplotlib inline`

This insures that the grahic is displayed in the notebook.

In [ ]:
# Using the example code from the Keras documentation
# See: https://keras.io/visualization/#training-history-visualization
# Plot training & validation accuracy values

import matplotlib.pyplot as plt
%matplotlib inline

plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

## Reducing overfitting
There are a few ways to reduce overfitting. It includes the use fo the `Dropout` layer and the `L1` and `L2` regularization functions.

We use the same model but try to regulate overfitting 

In [ ]:
from keras.layers import Dropout 

# Create a sequential model (most common in Keras)
classifier = Sequential()

# Create the first hidden layer
classifier.add(Dense(8, activation='relu', input_shape=(15,))) # there are 15 attributes
classifier.add(Dropout(0.2))

# Create the second hidden layer
classifier.add(Dense(8, activation='relu'))

# Create the output layer
classifier.add(Dense(1, activation='sigmoid'))

# Compile the neural network
classifier.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model (fit)
# batch size: number of records used in each epoch
history = classifier.fit(X_train, y_train, batch_size=50, epochs=100, validation_data=(X_test, y_test))

In [ ]:
# Using the example code from the Keras documentation
# See: https://keras.io/visualization/#training-history-visualization
# Plot training & validation accuracy values

import matplotlib.pyplot as plt
%matplotlib inline

plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

In [ ]:
from keras import regularizers

# Create a sequential model (most common in Keras)
classifier = Sequential()

# Create the first hidden layer
classifier.add(Dense(8, activation='relu', kernel_regularizer=regularizers.l2(0.03), input_shape=(15,))) # there are 15 attributes

# Create the second hidden layer
classifier.add(Dense(8, activation='relu'))

# Create the output layer
classifier.add(Dense(1, activation='sigmoid'))

# Compile the neural network
classifier.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model (fit)
# batch size: number of records used in each epoch
history = classifier.fit(X_train, y_train, batch_size=50, epochs=100, validation_data=(X_test, y_test))

In [ ]:
# Using the example code from the Keras documentation
# See: https://keras.io/visualization/#training-history-visualization
# Plot training & validation accuracy values

import matplotlib.pyplot as plt
%matplotlib inline

plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()